In [16]:
# config
inc='inc1'

In [31]:
# read files form data_dir/input_files directory and create a list of filepaths
def create_file_hm(inc):
    import os
    input_dir=f'./data_dir/input_files/{inc}'
    print(f"Files in the directory: {input_dir}")
    files = os.listdir(input_dir)
    # Filtering only the files.
    # files = [f for f in files if os.path.isfile(input_dir+'/'+f)]
    files = [f for f in files]
    files=[f'{input_dir}/{f}' for f in files]
    print(*files, sep="\n")
    # fd={}
    # for f in files:
    #     fd[f]=open(f'{input_dir}/{f}')
    # print(fd)
    return files
files=create_file_hm(inc)

Files in the directory: ./data_dir/input_files/inc1
./data_dir/input_files/inc1/mail1.txt
./data_dir/input_files/inc1/jackhammer.wav
./data_dir/input_files/inc1/mail2.txt
./data_dir/input_files/inc1/harvard.wav


In [18]:
# processing text files
def proc_txt_data(files):
    txt_files=[f for f in files if '.txt' in f]
    txt_data=""
    for tf in txt_files:
        print(f"reading {tf} file----------------")
        with open(tf) as f:
            txt_data+='\n'.join(f.readlines())
    print(txt_data)
    return txt_data
txt_data=proc_txt_data(files)

reading ./data_dir/input_files/inc1/mail1.txt file----------------
reading ./data_dir/input_files/inc1/mail2.txt file----------------
this mail contains summary about the incident 34567


In [29]:
# load azure keys
import json
with open('../keys/azure.json') as json_file:
    azure_keys = json.load(json_file)
print(azure_keys.keys())


dict_keys(['stt_key1', 'stt_key2', 'openai-key1', 'openai-key2'])


In [22]:
# extract data from azure speech to text api : convert speech to text
import sys
# %pip install moviepy
# from moviepy.editor import *

# %pip install azure==4.0.0
# % pip install azure-cognitiveservices-speech
import azure.cognitiveservices.speech as speechsdk


def proc_video_data(files):
    aud_files=[f for f in files if '.mp4' in f or '.wav' in f]
    
    aud_data=""
    for af in aud_files:
        speech_key, service_region = azure_keys['stt_key1'], "eastus"
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        audio_config = speechsdk.audio.AudioConfig(filename=af)
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
        result = speech_recognizer.recognize_once()
        print(result.text)
        aud_data+="\n"+result.text
    return aud_data

aud_data=proc_video_data(files)

The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham tacos. Al pastora are my favorite. A zestful food is the hot cross bun.
The stale smell of old beer lingers.
The stale smell of old beer lingers.
The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham tacos. Al pastora are my favorite. A zestful food is the hot cross bun.


In [53]:
# create summary from azure open ai api
def generate_summary(content):
    print("content to summarize----------------\n",content)
    prompt=f""" 
    summarize the content from below text data:{content}
    """
    from openai import AzureOpenAI
    from azure.identity import DefaultAzureCredential, get_bearer_token_provider
    deployment = "Test-GPT"
    endpoint="https://genai-warriors-openai-test.openai.azure.com/"
    # token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        # azure_ad_token_provider=token_provider,
        api_version="2024-02-01",
        api_key=azure_keys['openai-key1']
    )
    completion = client.completions.create(
        model=deployment,
        prompt=prompt
    )
    response=completion
    res=response.choices[0]


    print("\n\nsummary--------------------------\n",res)
    return res
   

content=txt_data+aud_data
summary=generate_summary(content)

content to summarize----------------
 this mail contains summary about the incident 34567
The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham tacos. Al pastora are my favorite. A zestful food is the hot cross bun.
The stale smell of old beer lingers.
The stale smell of old beer lingers.
The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham tacos. Al pastora are my favorite. A zestful food is the hot cross bun.


summary--------------------------
 CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' >>>>>>>>>>>>>>   summary    <<<<<<<<<<<<<<<<<< \nthis mail contains summary about the', content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': 